In [1]:
from PIL import Image
def format_data(sample):
    final = []

    img_path = sample["conversations"][0]["Image"]

    img = Image.open(img_path).convert("RGB")

    width = img.size[0]
    height = img.size[1]

    # final.append({
    #     "role": "system",
    #     "content": [
    #         {
    #             "type": "text",
    #             "text": SYSTEM_MESSAGE
    #         }            
    #     ]
    # })

    for conversation in sample["conversations"]:
        if conversation["from"] == "user":
            zoom = conversation["zoom"]
            new_x1 = int(zoom[0] * width)
            new_y1 = int(zoom[1] * height)
            new_x2 = int(zoom[2] * width)
            new_y2 = int(zoom[3] * height)

            cropped_img = img.crop((new_x1, new_y1, new_x2, new_y2))

            text = conversation["value"].split("\n")[1]

            final.append({
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": cropped_img,
                    },
                    {
                        "type": "text",
                        "text": text
                    }
                ]
            })
        else:
            final.append({
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": conversation["value"]
                    }
                ]
            })

    return final

In [2]:
import os

os.chdir('../')

In [3]:
print(os.getcwd())

/ocean/projects/cis240092p/amartin1/ReVL


In [2]:
import json
with open("../data/json_data/tmp_qwen2_test_data.json", "r") as f:
        train_dataset = json.load(f)

In [9]:
train_dataset = [format_data(sample) for sample in train_dataset]

In [10]:
train_dataset

[[{'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=1920x1080>},
    {'type': 'text',
     'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': '2'}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=960x540>},
    {'type': 'text',
     'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': '1'}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=480x270>},
    {'type': 'text',
     'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
  {'role': 'assistant', 'content': [{'

In [16]:
from transformers import AutoProcessor

processor2 = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

In [4]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B")


/ocean/projects/cis240092p/amartin1/ReVL/.qwen-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
processor.chat_template

"{% if messages is string %}{{ messages }}{% else %}{% for content in messages %}{% if content['type'] == 'image' or 'image' in content or 'image_url' in content %}<|vision_start|><|image_pad|><|vision_end|>{% elif content['type'] == 'video' or 'video' in content %}<|vision_start|><|video_pad|><|vision_end|>{% elif 'text' in content %}{{ content['text'] }}{% endif %}{% endfor %}{% endif %}"

In [ ]:
from jinja2 import Template

# Define your Jinja template as a string
template_str = """
Hello, {{ name }}!
You have {{ messages }} new messages.
"""

# Create a Template object
template = Template(template_str)

# Render the template with variables
rendered_output = template.render(name="Alice", messages=5)

print(rendered_output)

In [36]:
test2 = [{'role': 'system', 'content': [{'type': 'text', 'text': 'test'}]}] + train_dataset[0]

In [38]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-VL-7B")

In [40]:
test2

[{'role': 'system', 'content': [{'type': 'text', 'text': 'test'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': '2'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': '1'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': '2'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screen

In [41]:
def process_chat_template(conversation, add_generation_prompt=False):
    """
    Manually constructs a chat prompt string according to Qwen2-VL specifications.
    
    Args:
        conversation (list): A list of message dictionaries. Each message must have:
            - "role": One of "system", "user", or "assistant".
            - "content": Either a plain string or a list of segments.
              Each segment is a dict with:
                  - "type": "text", "image", or "video"
                  - For text segments: a "text" key with the message string.
                  - For image segments: an "image" key (its value is ignored here, a placeholder is used).
                  - For video segments: similarly a "video" key.
        add_generation_prompt (bool): If True, appends a new assistant prompt (i.e. "<|im_start|>assistant\n") at the end.
                                       
    Returns:
        str: The full formatted prompt string.
    """
    # Define the special tokens per Qwen2-VL's chat format.
    im_start = "<|im_start|>"
    im_end = "<|im_end|>"
    # Placeholders for non-text segments.
    vision_placeholder = "<|vision_start|><|image_pad|><|vision_end|>"
    video_placeholder = "<|vision_start|><|video_pad|><|vision_end|>"
    
    parts = []
    for message in conversation:
        role = message.get("role", "").strip().lower()
        # Begin the message block with role header.
        message_block = f"{im_start}{role}\n"
        content = message.get("content", "")
        
        # If content is a list of segments, process each segment.
        if isinstance(content, list):
            segment_texts = []
            for segment in content:
                seg_type = segment.get("type", "text").lower()
                if seg_type == "text":
                    text = segment.get("text", "")
                    segment_texts.append(text)
                elif seg_type == "image":
                    segment_texts.append(vision_placeholder)
                elif seg_type == "video":
                    segment_texts.append(video_placeholder)
                else:
                    segment_texts.append(str(segment))
            message_block += "\n".join(segment_texts)
        else:
            message_block += str(content)
            
        # End the message block.
        message_block += f"\n{im_end}\n"
        parts.append(message_block)
        
    if add_generation_prompt:
        parts.append(f"{im_start}assistant\n")
        
    return "".join(parts)


In [42]:
test_format = process_chat_template(test2)

In [44]:
print(test_format)

<|im_start|>system
test
<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>
In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?
<|im_end|>
<|im_start|>assistant
2
<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>
In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?
<|im_end|>
<|im_start|>assistant
1
<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>
In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?
<|im_end|>
<|im_start|>assistant
2
<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>
In this UI screenshot, what is the position of the element corresponding to the command "Shop" (with point)?
<|im_end|>
<|im_start|>assistant
(0.96, 0.32)
<|im_end|>



In [45]:
inputs  = processor(text=[test_format], images=images, return_tensors="pt")

In [46]:
inputs

{'input_ids': tensor([[151644,   8948,    198,  ...,    340, 151645,    198]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]), 'pixel_values': tensor([[-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        [-1.7923, -1.7923, -1.7923,  ..., -1.4802, -1.4802, -1.4802],
        ...,
        [-1.2959, -1.3251, -1.3397,  ..., -1.0678, -1.0394, -0.9541],
        [-1.1937, -1.1353, -1.0769,  ..., -0.9399, -0.9683, -1.0252],
        [-1.2667, -1.2813, -1.3105,  ..., -0.9399, -0.9683, -1.0252]]), 'image_grid_thw': tensor([[  1,  78, 138],
        [  1,  38,  68],
        [  1,  20,  34],
        [  1,  10,  18]])}

In [39]:
tokenizer.apply_chat_template(test2)

[]

In [47]:
processor.tokenizer.convert_tokens_to_ids(processor.image_token)

151655

In [31]:
images = []

for msg in test:
    for i, content in enumerate(msg["content"]):
        if content["type"] == "image":
            images.append(content["image"])
            msg["content"][i] = {"type": "image"}

test

[{'role': 'system', 'content': [{'type': 'text', 'text': 'test'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': '2'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': '1'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screenshot, what is the partition of the element corresponding to the command "Shop" (with quadrant number)?'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': '2'}]},
 {'role': 'user',
  'content': [{'type': 'image'},
   {'type': 'text',
    'text': 'In this UI screen

In [35]:

processor.apply_chat_template(test)

''